In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import KFold, cross_val_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import scale

In [2]:
names = ["Type", "Alcohol", "Malic acid" , "Ash", 'Alcalinity of ash', 'Magnesium',
         'Total phenols', 'Flavanoids', 'Nonflavanoid phenols',
         'Proanthocyanins', 'Color intensity', 'Hue', 'OD280/OD315 of diluted wines', 'Proline']

**1. Загрузите выборку Wine**

In [179]:
data = pd.read_csv("../../../data/wine.data")
first = data.columns 
data.columns = names

In [180]:
a_series = pd.Series(first, index = data.columns)
data = data.append(a_series, ignore_index=True)

C:\Users\Stanislav\AppData\Local\Temp\ipykernel_1412\4172409597.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append(a_series, ignore_index=True)


**2. Извлеките из данных признаки и классы. Класс записан в первом столбце (три варианта), признаки — в столбцах со второго по последний. Более подробно о сути признаков можно прочитать по адресу https://archive.ics.uci.edu/ml/datasets/Wine (см. также файл wine.names, приложенный к заданию)**

In [6]:
data = np.genfromtxt("../../../data/wine.data", delimiter=',')
y = data[:,0] 
X = data[:,1:]

In [3]:
data = pd.read_csv("../../../data/wine.data")
X = data[data.columns[1:]]
y = data[data.columns[0]]

In [181]:
X = data[data.columns[1:]]
y = data[data.columns[0]]

В зависимости от способа прочтения и значения "первого соседа" менялись результаты до нормировки.

**3. Оценку качества необходимо провести методом кросс-валидации по 5 блокам (5-fold). Создайте генератор разбиений, который перемешивает выборку перед формированием блоков (shuffle=True). Для воспроизводимости результата, создавайте генератор KFold с фиксированным параметром random_state=42. В качестве меры качества используйте долю верных ответов (accuracy).**

In [3]:
split = KFold(n_splits=5, shuffle=True, random_state=42)

**4. Найдите точность классификации на кросс-валидации для метода k ближайших соседей (sklearn.neighbors.KNeighborsClassifier), при k от 1 до 50. При каком k получилось оптимальное качество? Чему оно равно (число в интервале от 0 до 1)? Данные результаты и будут ответами на вопросы 1 и 2.**

In [4]:
def KNN_accuracy(n_neighbors_max, split, X, y):
    means = list()
    for k in range(1,n_neighbors_max+1):
        K_class = KNeighborsClassifier(n_neighbors=k, n_jobs=-1)  
        score = cross_val_score(estimator = K_class, X = X, 
y = y.astype('int'), cv=split,scoring='accuracy') 
        # в качестве y засовываем int/float и тд не object как если у нас датафрейм y.astype('int')
        means.append(np.mean(score))
    return means

In [6]:
means = KNN_accuracy(50,split,X,y)
means = pd.DataFrame(means,columns=['cross_val'])

In [192]:
means.max()

cross_val    0.735238
dtype: float64

**5. Произведите масштабирование признаков с помощью функции sklearn.preprocessing.scale. Снова найдите оптимальное k на кросс-валидации.**

In [7]:
X_train_scaled = scale(X)

In [10]:
scl = StandardScaler()
scl.fit(X)
X_train_scaled = scl.transform(X)

In [11]:
means = KNN_accuracy(50,split, X_train_scaled, y)
means = pd.DataFrame(means,columns=['cross_val'])
means.max()

cross_val    0.977619
dtype: float64

**6. Какое значение k получилось оптимальным после приведения признаков к одному масштабу? Приведите ответы на вопросы 3 и 4. Помогло ли масштабирование признаков?**

In [12]:
means.sort_values(by='cross_val').tail(5)

,cross_val
16,0.966508
17,0.966508
14,0.972063
15,0.972063
28,0.977619
